# Image 1

In [7]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def improve_image_contrast(input_image_path, clip_limit=3.0, tile_grid_size=(8, 8)):

    input_image = cv2.imread(input_image_path)

    # Convert the image to LAB color space
    lab_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2LAB)

    # Extract the L channel (luminance) from LAB
    luminance_channel = lab_image[:, :, 0]

    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) to the L channel
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    enhanced_luminance_channel = clahe.apply(luminance_channel)

    # Replace the original L channel with the enhanced one
    lab_image[:, :, 0] = enhanced_luminance_channel

    # Convert the enhanced LAB image back to BGR
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)


    cv2_imshow(input_image)
    cv2_imshow(enhanced_image)

    return enhanced_image
input_image_path = '/content/ShadowRemoval1.jpg'
improved_contrast_image = improve_image_contrast(input_image_path)


In [11]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from tqdm import tqdm

def process_shadow_reduction(input_image, luminance_thresholds=((0, 15), (45, 65)), neighborhood_size=1000):
    # Convert the input image from BGR to LAB color space
    lab_image_input = cv2.cvtColor(input_image, cv2.COLOR_BGR2LAB)

    # Extract the luminance channel (L) from LAB
    luminance_channel_input = lab_image_input[:, :, 0]

    # Create a binary mask for shadow regions based on luminance thresholds
    shadow_mask_input = np.zeros_like(luminance_channel_input, dtype=np.uint8)
    for lower, upper in luminance_thresholds:
        shadow_mask_input |= ((luminance_channel_input > lower) & (luminance_channel_input < upper)).astype(np.uint8) * 255

    # Invert the shadow mask to obtain the non-shadow regions
    non_shadow_mask_input = cv2.bitwise_not(shadow_mask_input)

    # Extract the neighborhood around each non-shadow pixel
    neighborhood_input = cv2.boxFilter(lab_image_input, -1, (neighborhood_size, neighborhood_size))

    # Calculate the average luminance of the neighborhood for each non-shadow pixel
    average_luminance_input = neighborhood_input / cv2.boxFilter(np.ones_like(lab_image_input), -1, (neighborhood_size, neighborhood_size))

    # Replace the shadow regions with the calculated average luminance
    result_image_lab = np.where(shadow_mask_input[..., None], average_luminance_input, lab_image_input)

    # Convert the result image back to BGR
    result_image_bgr = cv2.cvtColor(result_image_lab.astype(np.uint8), cv2.COLOR_LAB2BGR)

    cv2_imshow(input_image)
    cv2_imshow(shadow_mask_input)
    cv2_imshow(result_image_bgr)

# UNCOMMENT THIS TO RUN IT
# COMMENTED IT AS IT WAS TAKING EXTRA SPACE, AND ZIP FILE SIZE BECAME MORE THAN 20MB. So i commented the fucntion call here
# To run this, just uncomment it , upload the image and run this cell
#process_shadow_reduction(improved_contrast_image, luminance_thresholds=((0, 1), (35, 75)), neighborhood_size=500)

# Image 2

In [10]:
# input_image_path = '/content/ShadowRemoval1.jpg'
# input_image = cv2.imread(input_image_path)
# process_shadow_reduction(improved_contrast_image, luminance_thresholds=((0, 9), (55, 150)), neighborhood_size=500)

# Was taking space so commented . If you want to run it, just uncomment it